In [0]:
PROJECT_PATH = '/home/dobos/project/ga_isochrones/python:' + \
    '/home/dobos/project/ga_pfsspec_all/python:' + \
    '/home/dobos/project/pysynphot'

GRID_PATH = '/datascope/subaru/data/pfsspec/models/stellar/grid/phoenix/phoenix_HiRes'
FILTER_PATH = '/datascope/subaru/data/pfsspec/subaru/hsc/filters/fHSC-g.txt'

ARM = 'mr'

DETECTOR_PATH = '/datascope/subaru/data/pfsspec/subaru/pfs/arms/{}.json'.format(ARM)
PSF_PATH = '/datascope/subaru/data/pfsspec/subaru/pfs/psf/import/{}.2/pca.h5'.format(ARM)
SKY_PATH = '/datascope/subaru/data/pfsspec/subaru/pfs/noise/import/sky.see/{}/sky.h5'.format(ARM)
MOON_PATH = '/datascope/subaru/data/pfsspec/subaru/pfs/noise/import/moon/{}/moon.h5'.format(ARM)

# Radial Velocity fit

Demo code to perform maximum likelihood analysis of
radial velociy measurements between a spectrum and a
given template, drawn from the BOSZ models.

In [0]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
from scipy.interpolate import interp1d
import h5py as h5

In [0]:
# Allow load project as module
for p in reversed(PROJECT_PATH.split(':')):
    sys.path.insert(0, p)

## Load spectrum grid

In [0]:
from pfs.ga.pfsspec.core.grid import ArrayGrid
from pfs.ga.pfsspec.stellar.grid import ModelGrid
from pfs.ga.pfsspec.stellar.grid.bosz import Bosz
from pfs.ga.pfsspec.stellar.grid.phoenix import Phoenix

In [0]:
fn = os.path.join(GRID_PATH, 'spectra.h5')
grid = ModelGrid(Phoenix(), ArrayGrid)
grid.preload_arrays = False
grid.load(fn, format='h5')

## Simulated observation

In [0]:
from pfs.ga.pfsspec.core import Filter
from pfs.ga.pfsspec.sim.obsmod.pipelines import StellarModelPipeline
from pfs.ga.pfsspec.core import Physics
from pfs.ga.pfsspec.core.psf import PcaPsf
from pfs.ga.pfsspec.sim.obsmod import Detector
from pfs.ga.pfsspec.sim.obsmod import Sky
from pfs.ga.pfsspec.sim.obsmod import Moon

In [0]:
filt_hsc_g = Filter()
filt_hsc_g.read(FILTER_PATH)

psf = PcaPsf()
psf.load(PSF_PATH)

detector = Detector()
detector.load_json(DETECTOR_PATH)
detector.psf = psf

sky = Sky()
sky.load(SKY_PATH, format='h5')

moon = Moon()
moon.load(MOON_PATH, format='h5')

In [0]:
from pfs.ga.pfsspec.sim.obsmod.observations import PfsObservation

In [0]:
obs = PfsObservation()
obs.detector = detector
obs.sky = sky
obs.moon = moon

In [0]:
pp = StellarModelPipeline()

pp.model_res = grid.resolution or 150000
pp.mag_filter = filt_hsc_g
pp.observation = obs

In [0]:
def get_observation(rv=0.0, **kwargs):
    """
    Generate a spectrum and calculate the variance (sigma) of realistic observational error.
    """

    args = {
        'mag': 22,
        'seeing': 0.5,
        'exp_time': 15 * 60,
        'exp_count': 4 * 3,
        'target_zenith_angle': 0,
        'target_field_angle': 0.0,
        'moon_zenith_angle': 45,
        'moon_target_angle': 60,
        'moon_phase': 0.,
        'redshift': Physics.vel_to_z(rv) 
    }

    idx = grid.get_nearest_index(**kwargs)
    spec = grid.get_model_at(idx)
    pp.run(spec, **args)

    return idx, spec

def get_template(convolve=True, **kwargs):
    """
    Generate a noiseless template spectrum with same line spread function as the
    observation but keep the original, high-resolution binning.
    """

    # TODO: add template caching

    idx = grid.get_nearest_index(**kwargs)
    temp = grid.get_model_at(idx)
    temp.cont = None        # Make sure it's not passed around for better performance
    temp.mask = None

    if convolve:
        temp.convolve_psf(psf)

    return idx, temp


In [0]:
rv = 310.0
M_H = -0.5
T_eff = 5500
log_g = 1.0
a_M = 0.0

idx, spec = get_observation(rv=rv, M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M)
_, temp = get_template(M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M)

f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

ax.plot(spec.wave, spec.flux, '-', lw=0.3)
ax.plot(spec.wave, spec.flux_err, '-', lw=0.3)
ax.set_ylim(0, None)

f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

ax.plot(spec.wave, spec.flux + spec.flux_err * np.random.normal(size=spec.flux.shape), '-', lw=0.3)
ax.plot(spec.wave, spec.flux, '-', lw=0.3)
ax.set_xlim(0.99 * spec.wave.min(), 1.01 * spec.wave.max())
ax.set_ylim(0, np.quantile(spec.flux, 0.99) * 1.2)

## Radial velocity

In [0]:
from pfs.ga.pfsspec.stellar.rvfit import RVFit

In [0]:
rvfit = RVFit()
rvfit.grid = grid
rvfit.psf = psf

In [0]:
# Calculate log likelihood as a function of rv around the real value

rvv = np.linspace(-300, 300, 31) + rv
log_L, _, _ = rvfit.get_log_L(spec, temp, rv=rvv)

# Fit a Lorentz profile

pp, _ = rvfit.fit_lorentz(rvv, log_L)
lor = rvfit.lorentz(rvv, *pp)

# Run full fitting

best_rv = rvfit.fit_rv(spec, temp)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

ax.plot(rvv, log_L)
ax.plot(rvv, lor)
ax.axvline(rv, c='k')
ax.axvline(best_rv, c='r')